In [11]:
import pandas as pd
from pathlib import Path

# -------------------------------------------------------
# 경로 및 CSV 로드
# -------------------------------------------------------
BASE_DIR = Path("/workspace/nas203/ds_RehabilitationMedicineData/IDs/Kimjihoo/3_project_HCCmove/data")  # 기본 경로
CSV_PATH = BASE_DIR / "metadata_new3_sorted.csv"                                                         # CSV 파일 경로
meta = pd.read_csv(CSV_PATH)                                                                             # CSV 로드

# -------------------------------------------------------
# N01 / sample_data 내부의 is_train == False 만 필터링
# -------------------------------------------------------
subset = meta[
    meta["frame_path"].str.contains("N01|sample_data", case=False, na=False)  # frame_path에 N01 또는 sample_data 포함
    & (meta["is_train"] == False)                                             # 학습 데이터가 아닌 경우
].copy()                                                                      # 필터링 후 복사본 생성

print(f"[INFO] N01 / sample_data 내부 is_train == False 개수: {len(subset)}개\n")

# -------------------------------------------------------
# 각 keypoints 폴더 내 JSON 파일 개수 확인 및 원본 인덱스 표시
# -------------------------------------------------------
total_json = 0                                                                # JSON 총합 초기화
folder_summary = []                                                           # 폴더 요약 리스트 초기화

for original_idx, row in subset.iterrows():                                   # 원래 meta 인덱스 포함 순회
    kp_dir = BASE_DIR / str(row["keypoints_path"])                            # keypoints 폴더 경로
    kp_dir_full = str(kp_dir.resolve())                                       # 절대경로로 변환
    json_count = len(list(kp_dir.glob("*.json"))) if kp_dir.exists() else 0   # JSON 개수 계산
    folder_summary.append((original_idx, kp_dir_full, json_count))            # 인덱스 포함 요약 저장
    total_json += json_count                                                  # 총합 누적

    # ---------------------------------------------------
    # 각 row의 상세 출력 (원본 CSV 인덱스 포함)
    # ---------------------------------------------------
    print(f"[CSV index: {original_idx}] 영상: {row['video_path']}")
    print(f"    ▶ keypoints_path : {row['keypoints_path']}")
    print(f"    ▶ frame_path     : {row['frame_path']}")
    print(f"    ▶ is_train       : {row['is_train']}")
    print(f"    ▶ JSON 파일 수   : {json_count}\n")

# -------------------------------------------------------
# 폴더별 요약 출력
# -------------------------------------------------------
print("[INFO] N01 / sample_data 내부 False 폴더별 JSON 개수\n")
for idx, folder, count in folder_summary:
    print(f"[CSV {idx}] 📂 {folder}\n   → {count} JSON 파일\n")

print(f"\n✅ 총 폴더 수: {len(folder_summary)}개")
print(f"📊 JSON 총합: {total_json}개")


[INFO] N01 / sample_data 내부 is_train == False 개수: 40개

[CSV index: 0] 영상: 0_RAW_DATA/AI_dataset/N01/N01_Treatment/diagonal__biceps_curl.mp4
    ▶ keypoints_path : 2_KEYPOINTS/AI_dataset/N01/N01_Treatment/diagonal__biceps_curl
    ▶ frame_path     : 1_FRAME/AI_dataset/N01/N01_Treatment/diagonal__biceps_curl
    ▶ is_train       : False
    ▶ JSON 파일 수   : 2432

[CSV index: 13] 영상: 0_RAW_DATA/AI_dataset/N01/N01_Treatment/frontal__hip_flexion_extension_bobath_table.mp4
    ▶ keypoints_path : 2_KEYPOINTS/AI_dataset/N01/N01_Treatment/frontal__hip_flexion_extension_bobath_table
    ▶ frame_path     : 1_FRAME/AI_dataset/N01/N01_Treatment/frontal__hip_flexion_extension_bobath_table
    ▶ is_train       : False
    ▶ JSON 파일 수   : 647

[CSV index: 19] 영상: 0_RAW_DATA/AI_dataset/N01/N01_Treatment/frontal__slr_bobath_table.mp4
    ▶ keypoints_path : 2_KEYPOINTS/AI_dataset/N01/N01_Treatment/frontal__slr_bobath_table
    ▶ frame_path     : 1_FRAME/AI_dataset/N01/N01_Treatment/frontal__slr_bobath_tab

In [1]:
import pandas as pd
from pathlib import Path
import shutil

# -------------------------------------------------------
# 경로 설정
# -------------------------------------------------------
BASE_DIR = Path("/workspace/nas203/ds_RehabilitationMedicineData/IDs/Kimjihoo/3_project_HCCmove/data")
CSV_PATH = BASE_DIR / "metadata_new3_sorted.csv"
OUT_CSV = BASE_DIR / "metadata_interp_yolo.csv"

INTERP_DIR = BASE_DIR / "4_INTERP_DATA"        # 보간 데이터 저장 폴더
YOLO_DIR   = BASE_DIR / "5_YOLO"               # YOLO 변환본 폴더

# -------------------------------------------------------
# CSV 로드 및 필터링
# -------------------------------------------------------
meta = pd.read_csv(CSV_PATH)
subset = meta[
    meta["keypoints_path"].str.contains("AI_dataset/N01", case=False, na=False)
    & (meta["is_train"] == True)
].copy()

print(f"[INFO] N01 / is_train == True 데이터 개수: {len(subset)}개")

# -------------------------------------------------------
# 새 열 추가
# -------------------------------------------------------
meta["interp_json_path"] = ""
meta["yolo_pose_dir"] = ""
meta["yolo_pose_labels"] = ""

# -------------------------------------------------------
# N01 / True 항목만 복사 및 경로 업데이트
# -------------------------------------------------------
for idx, row in subset.iterrows():
    src_dir = BASE_DIR / str(row["keypoints_path"])  # 원본 keypoints 폴더
    rel_path = src_dir.relative_to(BASE_DIR / "2_KEYPOINTS")  # 2_KEYPOINTS 기준 상대경로

    interp_dst = INTERP_DIR / rel_path
    yolo_json_dst = YOLO_DIR / "YOLOv11_pose" / rel_path / "json"
    yolo_label_dst = YOLO_DIR / "YOLOv11_pose" / rel_path / "labels"

    # 보간 데이터 복사
    if not interp_dst.exists():
        print(f"[COPY] {src_dir} → {interp_dst}")
        shutil.copytree(src_dir, interp_dst)

    # metadata에 경로 저장
    meta.loc[idx, "interp_json_path"] = str(interp_dst)
    meta.loc[idx, "yolo_pose_dir"] = str(yolo_json_dst)
    meta.loc[idx, "yolo_pose_labels"] = str(yolo_label_dst)

print(f"\n✅ 복사 및 경로 업데이트 완료 ({len(subset)}개 항목).")

# -------------------------------------------------------
# 새로운 CSV 저장
# -------------------------------------------------------
meta.to_csv(OUT_CSV, index=False)
print(f"✅ metadata_interp_yolo.csv 저장 완료 → {OUT_CSV}")


[INFO] N01 / is_train == True 데이터 개수: 83개
[COPY] /workspace/nas203/ds_RehabilitationMedicineData/IDs/Kimjihoo/3_project_HCCmove/data/2_KEYPOINTS/AI_dataset/N01/N01_Treatment/diagonal__hip_extension → /workspace/nas203/ds_RehabilitationMedicineData/IDs/Kimjihoo/3_project_HCCmove/data/4_INTERP_DATA/AI_dataset/N01/N01_Treatment/diagonal__hip_extension
[COPY] /workspace/nas203/ds_RehabilitationMedicineData/IDs/Kimjihoo/3_project_HCCmove/data/2_KEYPOINTS/AI_dataset/N01/N01_Treatment/diagonal__hip_flexion_extension → /workspace/nas203/ds_RehabilitationMedicineData/IDs/Kimjihoo/3_project_HCCmove/data/4_INTERP_DATA/AI_dataset/N01/N01_Treatment/diagonal__hip_flexion_extension
[COPY] /workspace/nas203/ds_RehabilitationMedicineData/IDs/Kimjihoo/3_project_HCCmove/data/2_KEYPOINTS/AI_dataset/N01/N01_Treatment/diagonal__hip_flexion_extension_bobath_table → /workspace/nas203/ds_RehabilitationMedicineData/IDs/Kimjihoo/3_project_HCCmove/data/4_INTERP_DATA/AI_dataset/N01/N01_Treatment/diagonal__hip_flex

In [4]:
!pip list

Package                   Version
------------------------- --------------
addict                    2.4.0
anyio                     4.11.0
argon2-cffi               25.1.0
argon2-cffi-bindings      25.1.0
arrow                     1.3.0
asttokens                 2.0.5
astunparse                1.6.3
async-lru                 2.0.5
attrs                     23.1.0
babel                     2.17.0
backcall                  0.2.0
beautifulsoup4            4.12.2
bleach                    6.2.0
boltons                   23.0.0
brotlipy                  0.7.0
certifi                   2023.7.22
cffi                      1.15.1
chardet                   4.0.0
charset-normalizer        2.0.4
chumpy                    0.70
click                     8.0.4
comm                      0.2.3
conda                     23.9.0
conda-build               3.27.0
conda-content-trust       0.2.0
conda_index               0.3.0
conda-libmamba-solver     23.7.0
conda-package-handling    2.2.0
conda_package_s